# Cross-Validation

In [1]:
import numpy as np
import csv as csv
import matplotlib.pyplot as plt
import pandas as pd
import itertools
%matplotlib inline

## Evaluation mit Cross-Validation
Um verschiede Verfahren und Parameter möglichst ohne die Gefahr des overfitting evaluieren zu können, steht man immer vor dem Problem: Mit welchen Daten trainiere ich meine Verfahren und mit welchen teste ich? Offensichtlich hängt das Ergebnis der Evaluation stark von der konkreten Auswahl des Test- bzw. Trainingsdatensatzes ab. 

Eine in der Literatur etablierte Methode der systematischen Evaluation ist Cross-Validation (Link: [Cross-Validation](https://en.wikipedia.org/wiki/Cross-validation_%28statistics%29)). Die grundlegende Idee des k-Fold  Cross-Validation (Link: [k-fold cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)\#k-fold_cross-validation)) ist wie folgt: Die Gesamtmenge an Klassen-annotierten Datensätzen $T$ wird zufällig in $k$ gleich große Teilmengen (Folds) $T_1 \dots T_k$ aufgeteilt. Es werden $k$ Testiteration $i_1 \dots i_k$ durchgeführt. In jeder Iteration wird jeweils eine andere Teilmenge $T_i$ als Testdatensatz und die restlichen Daten $T \setminus T_i$ als Trainingsdatensatz verwendet. Als Gesamt Ergebniss der Cross-Validation wird der Mittelwert der Genauigkeiten der einzelnen Iteration herangezogen. 

Weitere Verfahren sind bspw. Holdout (Link: [Holdout](https://en.wikipedia.org/wiki/Cross-validation_(statistics)\#Holdout_method)), Nested cross-validation (Link: [Nested cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)\#Nested_cross-validation)) etc.

<figure>
<img src="./Figures/k-fold-cross-validation.png" alt="drawing" style="width:600px;">
    <figcaption>k-fold Cross Validation, Quelle: https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/K-fold_cross_validation_EN.svg/500px-K-fold_cross_validation_EN.svg.png
        </figcaption>
</figure>

Erweitern Sie Ihre Implementierung des KNN-Algorithmus aus dem vorherigen Teil um das <b>k-fold Cross-Validation</b> Verfahren. Wählen Sie hierbei einen geeigneten Wert für die Anzahl der k-folds, bzw. experimentieren Sie mit verschiedenen Werte.

In [2]:
DATA_FILE = './Data/original_titanic.csv'

df = pd.read_csv(DATA_FILE)

# (1) Datenlücken interpolieren
def prepareData(df):
    # TODO: implement
    df['Age'] = df.groupby(['Sex','Pclass', 'Survived'])['Age'].transform(lambda x:x.fillna(x.median()))
    return df

df_prepared = prepareData(df)

# (2) Datensatz stochastisch verändern
df_shuffled =  df.reindex(np.random.permutation(df.index)) # TODO: implement


In [13]:
# prepare one-hot-encodings for Sex, Pclass and Embarked
df_with_dummies = pd.get_dummies(df_shuffled, columns = ["Sex","Pclass","Embarked"], \
                                 prefix=["Sex_type","Pclass_type","Em_type"])

df_shuffled = df_with_dummies.sample(frac=1).reset_index(drop=True)

df_train=df_shuffled.sample(frac=0.8,random_state=1)
df_test=df_shuffled.drop(df_train.index)

In [14]:
# feature : which features to keep
def extractFeatureVector(row):
    return [row['Age'], row['Fare'], row['Sex_type_female'], row['Sex_type_male'],\
            row['Pclass_type_1'], row['Pclass_type_2'], row['Pclass_type_3'],\
            row['Em_type_C'], row['Em_type_Q'], row['Em_type_S']]

In [15]:
class KNN(object):
    
    def __init__(self, k):
        self.k = k

    def distance(self, vector1, vector2):
        return np.sqrt(np.nansum((vector1 - vector2)**2))
    
    
    def fit(self, df):
        trainSet = []
        for i in range (len(df)):
            trainSet.append(extractFeatureVector(df.iloc[i]))
    
        self.trainData = np.array(trainSet)
        self.trainLabel = np.array(df['Survived'])

    
    def _euclidean_dist(x, y):
        return 

    def predict(self, x):
        dist_to_train_sample = [(self.distance(x, train_sample), i) for i, train_sample in enumerate(self.trainData)]
        sorted_dist = sorted(dist_to_train_sample, key=lambda x: x[0])
        
        k_nearest = sorted_dist[:self.k]
#         print(k_nearest)
        
        k_nearest_labels = [self.trainLabel[i] for _, i in k_nearest]
#         print(k_nearest_labels)
        
        ones = (np.sum(np.array(k_nearest_labels)))
        zeros = self.k - ones

        #tie breaker, return random
        if ones == zeros:
            return np.random(k_nearest_labels)
        
        return 1 if (ones > zeros) else 0

In [5]:
len(df_shuffled)

1309

In [16]:
def calculate_normalize(df, df_train):
    df_copy = df.copy()
    df_copy['Age'] = (df['Age'] - np.mean(df_train['Age']))/np.std(df_train['Age'])
    df_copy['Fare'] = (df['Fare'] - np.mean(df_train['Fare']))/np.std(df_train['Fare'])
    return df_copy

In [17]:
knn = KNN(k=5)

iteration = 11
len_test = int(len(df_shuffled)/iteration)
classification_rates = []             

for i in range(iteration):
    df_test = df_shuffled[len_test*i :len_test*i + len_test]
    if i != 0:
        part1 = df_shuffled[:len_test*i]
        part2 = df_shuffled[len_test*i + len_test:]
        df_train = pd.concat([part1, part2])
    else:
        df_train = df_shuffled[len_test*i + len_test:]
        
    df_train_normed = calculate_normalize(df_train, df_train)    
    df_test_normed = calculate_normalize(df_test, df_train)
#     print(f'iteration {i+1}/{iteration}')
    
    knn.fit(df_train_normed)
    correct_prediction = 0
    for i in range(len(df_test)):
        survival_prediction = knn.predict(extractFeatureVector(df_test_normed.iloc[i]))
        if survival_prediction == df_test_normed.iloc[i]['Survived']:
            correct_prediction += 1

    classification_rates.append(correct_prediction / len(df_test))

In [12]:
df_train_normed.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Boat', 'Body',
       'Home-Dest'],
      dtype='object')

In [18]:
print(classification_rates)

[0.7815126050420168, 0.8235294117647058, 0.865546218487395, 0.7983193277310925, 0.7899159663865546, 0.8235294117647058, 0.7899159663865546, 0.8235294117647058, 0.865546218487395, 0.7563025210084033, 0.8235294117647058]


In [19]:
np.mean(classification_rates)

0.8128342245989305